::## ANI Model Training with stress term and transfer learning



### Step 1: Installing TorchANI

We need to install TorchANI.

The original TorchANI implementation does not include functionality to handle varying cell sizes during training.

For this tutorial, we will use a customized version of TorchANI that supports this feature, which needs to be downloaded and installed/



In [1]:
!pip install git+https://github.com/gsjung0419/torchani_gs.git@v
!pip install pyarmor
!apt-get install -y hdf5-tools

  Cloning https://github.com/gsjung0419/torchani_gs.git (to revision v) to /tmp/pip-req-build-apo7mown
  Running command git clone --filter=blob:none --quiet https://github.com/gsjung0419/torchani_gs.git /tmp/pip-req-build-apo7mown
  Running command git checkout -b v --track origin/v
  Switched to a new branch 'v'
  Branch 'v' set up to track remote branch 'v' from 'origin'.
  Resolved https://github.com/gsjung0419/torchani_gs.git to commit 0c759d4249812f5c2f3f012a72642f587950e08b
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
hdf5-tools is already the newest version (1.10.7+repack-4ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


### Step 2: Setting up File Path

Connect google drive and move to the path where you downloaded the tutorial.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/ANI_Tutorials/Chapter2-Stress_Term
import sys
sys.path.append('./dist')  # Replace with the actual path to the `dist` folder


/content/drive/MyDrive/ANI_Tutorials/Chapter2-Stress_Term


### Step 3: Importing Required Libraries

In this step, we import the necessary libraries for our task.

In the Runtime menu, click 'change runtime type', you may need to restart the session and download the torchani again.

Initialize any save parameter file (*.pt)

In [4]:
import torch
import torchani
import os
import math
import tqdm
import wandb
from Atutils import *

/usr/local/lib/python3.11/dist-packages/torchani/aev.py:17: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")
/usr/local/lib/python3.11/dist-packages/torchani/__init__.py:55: UserWarning: Dependency not satisfied, torchani.ase will not be available
  warnings.warn("Dependency not satisfied, torchani.ase will not be available")


In [5]:
!rm *.pt
!ls

Atomic_Train.ipynb  __pycache__  sae_linfit_dftb.data  train.h5       wandb
dist		    rC.params	 tot.h5		       validation.h5


###Step 4: Hyper Parameter Setting

We utilize WandB. If you want to use other (e.g., tensorboard), you may want to check the first tutorial.

For WandB, you need to sign up and copy the API key.

In [6]:
#This code requires to login wandb
wandb.init(project="Stress-Term-Train1", config={
    "force_coefficient": 0.1,
    "virial_coefficient": 0.025,
    "energy_coefficient": 1.0,
    "initial_lr": 0.001,
    "batch_size": 64,
    "max_epochs": 400,
})

# Load hyperparameters from WandB config
energy_coefficient = wandb.config.energy_coefficient
force_coefficient = wandb.config.force_coefficient
virial_coefficient = wandb.config.virial_coefficient
max_epochs = wandb.config.max_epochs
batch_size = wandb.config.batch_size

wandb: Currently logged in as: gsjung. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


### Step 5: Loading ANI Parameters.

Please use the edfault version.

You may check the original paper for the symmetry function, and how AEV computing work.

Please decide where to run. You should provide the path of google drive where you are running this tutorial.

In [7]:
# helper function to convert energy unit from Hartree to kcal/mol
from torchani.units import hartree2kcalmol

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
###############################################################################
# Now let's read self energies and construct energy shifter.
try:
    path = os.path.dirname(os.path.realpath(__file__))
except NameError:
    path = os.getcwd()

sae_file = os.path.join(path, 'sae_linfit_dftb.data')  # noqa: E501
const_file = os.path.join(path, 'rC.params')

consts = torchani.neurochem.Constants(const_file)
aev_computer = torchani.AEVComputer(**consts)

energy_shifter = torchani.neurochem.load_sae(sae_file)
species_order = ['C']



### Step 6: Data Preprocessing

In this step, we define the batchsize.

You can change and test.  

Training/Validation sets are already given

In [8]:
trpath = os.path.join(path, 'train.h5')
valpath = os.path.join(path, 'validation.h5')

training = torchani.data.load(
    trpath,
    additional_properties=('forces','cell','virial')
).subtract_self_energies(energy_shifter, species_order).species_to_indices(species_order).shuffle()

validation = torchani.data.load(
    valpath,
    additional_properties=('forces','cell','virial')
).subtract_self_energies(energy_shifter, species_order).species_to_indices(species_order).shuffle()


training = training.collate(batch_size).cache()
validation = validation.collate(batch_size).cache()

Please install hdf5-tools to check the data "*.h5"

In [9]:
!h5ls -rf train.h5

/                        Group
/mols                    Group
/mols/mol                Group
/mols/mol/cell           Dataset {2004, 3, 3}
/mols/mol/coordinates    Dataset {2004, 32, 3}
/mols/mol/energies       Dataset {2004}
/mols/mol/forces         Dataset {2004, 32, 3}
/mols/mol/species        Dataset {32}
/mols/mol/virial         Dataset {2004, 6}


Data loading for batch trainning.

Note that additional_properties you can define to load it in addition to 'energy,'  and 'coordinates'.

Here, we define PBC and AEV dimensions and neural network structure (See Slide)

In [10]:
pbc = torch.tensor([1,1,1],dtype=torch.bool,device=device)
###############################################################################
# The code to define networks, optimizers, are mostly the same
aev_dim = aev_computer.aev_length
C_network = create_C_network(aev_dim)
nn = torchani.ANIModel([C_network])


Here, we initialize all the parameters.

In [11]:
def init_params(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight, a=1.0)
        torch.nn.init.zeros_(m.bias)

nn.apply(init_params)


ANIModel(
  (0): Sequential(
    (0): Linear(in_features=54, out_features=224, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=224, out_features=192, bias=True)
    (3): GELU(approximate='none')
    (4): Linear(in_features=192, out_features=160, bias=True)
    (5): GELU(approximate='none')
    (6): Linear(in_features=160, out_features=1, bias=True)
  )
)

### Step 7: Training the Model

Here we define scheduler.

For the weights, we utilize Adam Optimizer.
For the bias, Stochastic Gradient Descendent (SGD) Algorithm is utilized.


In [12]:
model = torchani.nn.Sequential(aev_computer, nn).to(device)

# Define learning rates and weight decays for each layer
learning_rates = [0.001, 0.001, 0.001, 0.001]
weight_decays = [0.0001, 0.0001, 0.0001, 0.0001]

# Configure a single AdamW optimizer for both networks
AdamW = configure_adamw_for_multiple(
    networks=[C_network],
    lrs=learning_rates,
    weight_decays=weight_decays
)

# Configure a single SGD optimizer for biases in both networks
SGD = configure_sgd_for_biases(
    networks=[C_network],
    lrs=learning_rates
)

AdamW_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(AdamW, factor=0.5, patience=100, threshold=0)
SGD_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(SGD, factor=0.5, patience=100, threshold=0)

Define check points and the file name for the parameters.

In [13]:
latest_checkpoint = 'training-latest.pt'
if os.path.isfile(latest_checkpoint):
    checkpoint = torch.load(latest_checkpoint)
    nn.load_state_dict(checkpoint['nn'])
    AdamW.load_state_dict(checkpoint['AdamW'])
    SGD.load_state_dict(checkpoint['SGD'])
    AdamW_scheduler.load_state_dict(checkpoint['AdamW_scheduler'])
    SGD_scheduler.load_state_dict(checkpoint['SGD_scheduler'])


Define the variable needed for validate() function to measure the performance of the trained model for the validation sets during the training.

In [14]:
# Initialize ValidationContext
context = ValidationContext(
    model=model,
    validation=validation,
    aev_computer=aev_computer,
    nn=nn,
    energy_shifter=energy_shifter,
    hartree2kcalmol=hartree2kcalmol,
    device=device,
    pbc=pbc
)

Model train!!!!

In [15]:
print("training starting from epoch", AdamW_scheduler.last_epoch + 1)
early_stopping_learning_rate = 1.0E-5

best_model_checkpoint = 'training-best.pt'
wandb.watch(model,log='all')

mse_sum = torch.nn.MSELoss(reduction='sum')
mse = torch.nn.MSELoss(reduction='none')

bestout=[0,0,0,0]
for epoch in range(AdamW_scheduler.last_epoch + 1, max_epochs):
    rmse_e,rmse_f,rmse_v= validate_woa(context)
    print('RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy:', rmse_e, rmse_f, rmse_v,'at epoch', AdamW_scheduler.last_epoch + 1)

    #which rmse is reference:
    rmse = rmse_e #best total energy

    learning_rate = AdamW.param_groups[2]['lr']
    if learning_rate < early_stopping_learning_rate:
        break

    # checkpoint
    if AdamW_scheduler.is_better(rmse, AdamW_scheduler.best):
        torch.save(nn.state_dict(), best_model_checkpoint)
        bestout=[rmse_e,rmse_f,rmse_v]

    AdamW_scheduler.step(rmse)
    SGD_scheduler.step(rmse)

    # Log epoch-level metrics
    wandb.log({
      "epoch": epoch + 1,
      "validation_rmse_energy": rmse_e,
      "validation_rmse_force": rmse_f,
      "validation_rmse_virial": rmse_v,
      "learning_rate": learning_rate,
    })

    for i, properties in tqdm.tqdm(
        enumerate(training),
        total=len(training),
        desc="epoch {}".format(AdamW_scheduler.last_epoch)
    ):

        energy_loss,force_loss,virial_loss, =compute_loss_woa(context,properties)

        loss = energy_coefficient*energy_loss + force_coefficient*force_loss + virial_coefficient*virial_loss

        AdamW.zero_grad()
        SGD.zero_grad()
        loss.backward()
        AdamW.step()
        SGD.step()

        #batch level metric
        wandb.log({
            "batch_loss": math.log(loss.item()),
            "energy_loss": math.log(energy_loss.item()),
            "force_loss": math.log(force_loss.item()),
            "virial_loss": math.log(virial_loss.item()),
        })

    torch.save({
        'nn': nn.state_dict(),
        'AdamW': AdamW.state_dict(),
        'SGD': SGD.state_dict(),
        'AdamW_scheduler': AdamW_scheduler.state_dict(),
        'SGD_scheduler': SGD_scheduler.state_dict(),
    }, latest_checkpoint)

print("Best model evalutaion.")
print("Total Energy (meV/atom): ",bestout[0]*0.0433634*1000/32)
print("Forces (meV/A): ",bestout[1]*0.0433634*1000)
print("Virial/Pressure*Volume (meV): ",bestout[2]*0.0433634*1000)

training starting from epoch 1
RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 2745.7229285422973 105.58602485421093 2176.8975223770435 at epoch 1


epoch 1: 100%|██████████| 32/32 [00:02<00:00, 14.63it/s]


RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 88.56791325191567 34.35125644463576 119.1199557020547 at epoch 2


epoch 2: 100%|██████████| 32/32 [00:02<00:00, 15.92it/s]


RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 39.362516724814284 28.97458280582452 46.81365089257497 at epoch 3


...

RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 2.4669474148576187 1.662946274545388 3.423748038676719 at epoch 398


epoch 398: 100%|██████████| 32/32 [00:02<00:00, 13.66it/s]


RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 22.307094904255592 1.6947103089519209 5.933337937035856 at epoch 399


epoch 399: 100%|██████████| 32/32 [00:02<00:00, 15.43it/s]

Best model evalutaion.
Total Energy (meV/atom):  2.3642030988802922
Forces (meV/A):  73.04898341023895
Virial/Pressure*Volume (meV):  141.75069569800942


### Step 8: Transfer Learning Setting

Here we define scheduler.

For the weights, we utilize Adam Optimizer.
For the bias, Stochastic Gradient Descendent (SGD) Algorithm is utilized.



In [16]:


# Define learning rates and weight decays for each layer
learning_rates = [0.0000, 0.0001, 0.0001, 0.000]
weight_decays = [0.0, 0.00001, 0.00001, 0.0]

# Configure a single AdamW optimizer for both networks
AdamW = configure_adamw_for_multiple(
    networks=[C_network],
    lrs=learning_rates,
    weight_decays=weight_decays
)

# Configure a single SGD optimizer for biases in both networks
SGD = configure_sgd_for_biases(
    networks=[C_network],
    lrs=learning_rates,
)

AdamW_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(AdamW, factor=0.8, patience=50, threshold=0)
SGD_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(SGD, factor=0.8, patience=50, threshold=0)

Initialization of parameter from the best model

In [17]:
nnt = torchani.ANIModel([C_network])
nnt.load_state_dict(torch.load('training-best.pt'))
model = torchani.nn.Sequential(aev_computer, nnt).to(device) #new model from the best parameters

<ipython-input-17-33bac1a575e0>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  nnt.load_state_dict(torch.load('training-best.pt'))


In [18]:
latest_checkpoint = 'transfer-latest.pt'
if os.path.isfile(latest_checkpoint):
    checkpoint = torch.load(latest_checkpoint)
    nnt.load_state_dict(checkpoint['nn'])
    AdamW.load_state_dict(checkpoint['AdamW'])
    SGD.load_state_dict(checkpoint['SGD'])
    AdamW_scheduler.load_state_dict(checkpoint['AdamW_scheduler'])
    SGD_scheduler.load_state_dict(checkpoint['SGD_scheduler'])


Trainning and Validation set are the entire data.

In [19]:
trpath = os.path.join(path, 'tot.h5')
valpath = os.path.join(path, 'tot.h5')

training = torchani.data.load(
    trpath,
    additional_properties=('forces','cell','virial')
).subtract_self_energies(energy_shifter, species_order).species_to_indices(species_order).shuffle()

validation = torchani.data.load(
    valpath,
    additional_properties=('forces','cell','virial')
).subtract_self_energies(energy_shifter, species_order).species_to_indices(species_order).shuffle()


training = training.collate(batch_size).cache()
validation = validation.collate(batch_size).cache()

New setting, focusing on forces and virial term.

These settup can be scheduled in the previous trainning, but this transfer learning make the fining tuning less complicated.

In [20]:
max_epochs_trans = 1000
early_stopping_learning_rate = 1.0E-9

force_coefficient = 0.4  # controls the importance of energy loss vs force loss
energy_coefficient = 0.2  # controls the importance of energy loss vs force loss
virial_coefficient = 0.4

In [21]:
# Initialize ValidationContext
context_transfer = ValidationContext(
    model=model,
    validation=validation,
    aev_computer=aev_computer,
    nn=nnt,
    energy_shifter=energy_shifter,
    hartree2kcalmol=hartree2kcalmol,
    device=device,
    pbc=pbc
)

Transfer Learning for model!!!

In [22]:
print("training starting from epoch", AdamW_scheduler.last_epoch + 1)
early_stopping_learning_rate = 1.0E-5

best_model_checkpoint = 'transfer-best.pt'
wandb.watch(model,log='all')

mse_sum = torch.nn.MSELoss(reduction='sum')
mse = torch.nn.MSELoss(reduction='none')

bestout=[0,0,0,0]
for epoch in range(AdamW_scheduler.last_epoch + 1, max_epochs_trans):
    rmse_e,rmse_f,rmse_v= validate_woa(context_transfer)
    print('RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy:', rmse_e, rmse_f, rmse_v,'at epoch', AdamW_scheduler.last_epoch + 1)

    #which rmse is reference:
    rmse = rmse_e #best total energy

    learning_rate = AdamW.param_groups[2]['lr']
    if learning_rate < early_stopping_learning_rate:
        break

    # checkpoint
    if AdamW_scheduler.is_better(rmse, AdamW_scheduler.best):
        torch.save(nn.state_dict(), best_model_checkpoint)
        bestout=[rmse_e,rmse_f,rmse_v]

    AdamW_scheduler.step(rmse)
    SGD_scheduler.step(rmse)

    # Log epoch-level metrics
    wandb.log({
      "epoch": epoch + 1,
      "validation_rmse_energy": rmse_e,
      "validation_rmse_force": rmse_f,
      "validation_rmse_virial": rmse_v,
      "learning_rate": learning_rate,
    })

    for i, properties in tqdm.tqdm(
        enumerate(training),
        total=len(training),
        desc="epoch {}".format(AdamW_scheduler.last_epoch)
    ):

        energy_loss,force_loss,virial_loss, =compute_loss_woa(context_transfer,properties)

        loss = energy_coefficient*energy_loss + force_coefficient*force_loss + virial_coefficient*virial_loss

        AdamW.zero_grad()
        SGD.zero_grad()
        loss.backward()
        AdamW.step()
        SGD.step()

        #batch level metric
        wandb.log({
            "batch_loss": math.log(loss.item()),
            "energy_loss": math.log(energy_loss.item()),
            "force_loss": math.log(force_loss.item()),
            "virial_loss": math.log(virial_loss.item()),
        })

    torch.save({
        'nn': nn.state_dict(),
        'AdamW': AdamW.state_dict(),
        'SGD': SGD.state_dict(),
        'AdamW_scheduler': AdamW_scheduler.state_dict(),
        'SGD_scheduler': SGD_scheduler.state_dict(),
    }, latest_checkpoint)

print("Best model evalutaion.")
print("Total Energy (meV/atom): ",bestout[0]*0.0433634*1000/32)
print("Forces (meV/A): ",bestout[1]*0.0433634*1000)
print("Virial/Pressure*Volume (meV): ",bestout[2]*0.0433634*1000)

training starting from epoch 1
RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 1.9700304590317506 1.8631966389180157 3.505753764519095 at epoch 1


epoch 1: 100%|██████████| 40/40 [00:02<00:00, 15.69it/s]


RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 7.1380263789332234 1.9427455881457754 3.566745906327676 at epoch 2


epoch 2: 100%|██████████| 40/40 [00:02<00:00, 14.82it/s]


RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 2.4626981593325468 1.821173098262788 2.863532734945949 at epoch 3


...

RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 1.8263183586922518 0.9703638119181144 1.5851992958444057 at epoch 569


epoch 569: 100%|██████████| 40/40 [00:02<00:00, 15.24it/s]


RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 1.8277273009428447 0.9700461968784259 1.5851759892864803 at epoch 570


epoch 570: 100%|██████████| 40/40 [00:02<00:00, 15.50it/s]


RMSE (kcal/mol) :Energy, Force, Virial, and Atomic Energy: 1.330113670519639 0.9647361180791909 1.5528820776268517 at epoch 571
Best model evalutaion.
Total Energy (meV/atom):  1.0682529538796766
Forces (meV/A):  49.53120185250497
Virial/Pressure*Volume (meV):  88.89665856316437



If you think this tutorial is useful, consider cite the following references:

References
1. The function for different cell sizes was developed in this study:

  Jung, G. S., Myung, H., & Irle, S. (2023). Artificial neural network potentials for mechanics and fracture dynamics of two-dimensional crystals. Machine Learning: Science and Technology, 4(3), 035001.

2. The function for different cell sizes for pressure/stres terms was developed in this study:

  Jung, G. S. (2024), Enhancing high-fidelity neural network potentials through low-fidelity sampling. APL Machine Learning, 2, 046102.

3. The data set were prepared by the study:

 Jung, G. S., Lee, S., & Choi, J. Y. (2023). Data Distillation for Neural Network Potentials toward Foundational Dataset. 37th Conference on Neural Information Processing Systems (NeurIPS 2023), arXiv preprint arXiv:2311.05407.

 using active learning tool developed by the study:
 Jung, G. S., Choi, J. Y., & Lee, S. M. (2024). Active learning of neural network potentials for rare events. Digital Discovery.

3. The original code of TorchANI developed by the study:

  Gao, X., Ramezanghorbani, F., Isayev, O., Smith, J. S., & Roitberg, A. E. (2020). TorchANI: A free and open source PyTorch-based deep learning implementation of the ANI neural network potentials. Journal of chemical information and modeling, 60(7), 3408-3415.

